# Zillow Challenge

##  Data input

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import seterr,isneginf,array
from datetime import datetime
from pandas import compat
from operator import itemgetter
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.neighbors import KDTree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)
pd.options.display.float_format = '{:20,.2f}'.format
compat.PY3 = False

def Decision_Tree_Image(decision_tree, feature_names, name="temp"):
    
    # Etrainport our decision tree to graphviz format
    dot_file = tree.export_graphviz(decision_tree, out_file='images/' + name + '.dot', feature_names=feature_names)
    
    # Call graphviz to make an image file from our decision tree
    os.system("dot -T png images/" + name + ".dot -o images/" + name + ".png")
    
    # Return the .png image so we can see it
    return Image(filename='images/' + name + '.png')

In [2]:
train_f = "/Users/Jose/Desktop/Zillow/train_2016_v2.csv"
train = pd.read_csv(train_f, index_col = 'parcelid', parse_dates=['transactiondate'])
train['transactiondate'] = pd.to_datetime(train['transactiondate']).astype(int)

properties_f = "/Users/Jose/Desktop/Zillow/properties_2016.csv"
properties = pd.read_csv(properties_f, index_col = 'parcelid')


train = train.join(properties) 
train_bak = train  # backup 

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#### TEST MODE  #####
train = train_bak
train = train.sample(frac=1)

##  Feature Analysis

In [4]:
# Add taxrate
train['taxrate'] = np.divide(train['taxamount'],train['taxvaluedollarcnt'])

In [5]:
# Add roomcnt_offby
train['roomcnt_offby'] = np.where(
                                train['roomcnt']==0,
                                0,
                                train['roomcnt'].add(train['bathroomcnt'])
                            )

In [6]:
# Clean features
for col in train:
    dt = train[col].dtype
    if dt == 'float64':
        train[col].fillna(-1, inplace='True')  
    elif dt == 'object':
        train = pd.get_dummies(train,columns=[col], dummy_na=True)

In [7]:
#train.query('logerror<-2 & logerror > -2.5')

##  Feature selection

In [8]:
Y = train[['logerror']]
X = train.drop('logerror', axis=1)

In [13]:
X.describe()

transactiondate  airconditioningtypeid  \
count                    90,275.00              90,275.00   
mean  1,465,669,839,260,003,584.00                  -0.10   
std       7,410,679,638,771,986.00                   2.13   
min   1,451,606,400,000,000,000.00                  -1.00   
25%   1,459,814,400,000,000,000.00                  -1.00   
50%   1,465,862,400,000,000,000.00                  -1.00   
75%   1,471,564,800,000,000,000.00                   1.00   
max   1,483,056,000,000,000,000.00                  13.00   

       architecturalstyletypeid         basementsqft          bathroomcnt  \
count                 90,275.00            90,275.00            90,275.00   
mean                      -0.98                -0.66                 2.28   
std                        0.47                18.22                 1.00   
min                       -1.00                -1.00                 0.00   
25%                       -1.00                -1.00                 2.00   
50%                       -1.00                -1.00                 2.00   
75%                       -1.00                -1.00                 3.00   
max                       21.00             1,555.00                20.00   

                bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
count            90,275.00            90,275.00              90,275.00   
mean                  3.03                -1.00                   3.17   
std                   1.16                 0.07                   3.50   
min                   0.00                -1.00                  -1.00   
25%                   2.00                -1.00                  -1.00   
50%                   3.00                -1.00                   4.00   
75%                   4.00                -1.00                   7.00   
max                  16.00                 4.00                  12.00   

         calculatedbathnbr           decktypeid  finishedfloor1squarefeet  \
count            90,275.00            90,275.00                 90,275.00   
mean                  2.27                -0.51                    101.45   
std                   1.04                 5.70                    400.03   
min                  -1.00                -1.00                     -1.00   
25%                   2.00                -1.00                     -1.00   
50%                   2.00                -1.00                     -1.00   
75%                   3.00                -1.00                     -1.00   
max                  20.00                66.00                  7,625.00   

       calculatedfinishedsquarefeet  finishedsquarefeet12  \
count                     90,275.00             90,275.00   
mean                       1,760.20              1,654.93   
std                          937.05                966.94   
min                           -1.00                 -1.00   
25%                        1,176.00              1,118.00   
50%                        1,535.00              1,476.00   
75%                        2,089.00              2,013.00   
max                       22,741.00             20,013.00   

       finishedsquarefeet13  finishedsquarefeet15  finishedsquarefeet50  \
count             90,275.00             90,275.00             90,275.00   
mean                  -0.49                 93.00                102.02   
std                   26.95                509.94                404.49   
min                   -1.00                 -1.00                 -1.00   
25%                   -1.00                 -1.00                 -1.00   
50%                   -1.00                 -1.00                 -1.00   
75%                   -1.00                 -1.00                 -1.00   
max                1,584.00             22,741.00              8,352.00   

       finishedsquarefeet6                 fips         fireplacecnt  \
count            90,275.00            90,275.00            90,275.00   
mean                  9.74             6,048

In [10]:
Y.describe()

,logerror
count,"4,514.00"
mean,0.01
std,0.15
min,-2.31
25%,-0.03
50%,0.01
75%,0.04
max,2.35


## Modeling

In [32]:
Y_train['logerror'].describe()

count              36,110.00
mean                    0.01
std                     0.15
min                    -4.61
25%                    -0.03
50%                     0.01
75%                     0.04
max                     3.44
Name: logerror, dtype: float64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.05)

//anaconda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [1]:
# Fit regression model
max_depth = None
min_samples_split = int(round(len(X_train)/50, 0))
min_samples_leaf = int(round(len(X_train)/20, 0))
print("min_samples_split =", min_samples_split)
print("min_samples_split =", min_samples_leaf)

model = DecisionTreeRegressor(max_depth=max_depth, 
                              min_samples_split = min_samples_split,
                              min_samples_leaf = min_samples_leaf,
                              criterion = "mae")
model.fit(X_train, Y_train)

NameError: name 'X_train' is not defined

In [11]:
Y_predict=model.predict(X_test)

In [29]:
# (<= value) left = false, right = true
zip(X.columns[model.tree_.feature], model.tree_.threshold, model.tree_.children_left, model.tree_.children_right)

[('calculatedfinishedsquarefeet', 2044.5, 1, 12),
 ('taxamount', 1944.945068359375, 2, 3),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxrate', 0.015335844829678535, 4, 11),
 ('transactiondate', 1.4727312290978202e+18, 5, 10),
 ('finishedsquarefeet12', 1444.5, 6, 9),
 ('censustractandblock', 60375512907776.0, 7, 8),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('transactiondate', 1.4683247988105544e+18, 13, 14),
 ('taxdelinquencyflag_Y', -2.0, -1, -1),
 ('taxdelinquencyflag_Y', -2.0, -1, -1)]

In [30]:
#Decision_Tree_Image(model, X_train.columns)

## Model evaluation

In [12]:
# Zillow MAE (test)
print("MAE: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

('MAE: ', 0.11135898066742846)


## Model Execution

In [469]:
sample_submission_f = '/Users/Jose/Desktop/Zillow/sample_submission.csv'
submission = pd.read_csv(sample_submission_f, index_col='ParcelId')

In [470]:
#  ATTEMPT 2 - 
logerroravg = train['logerror'].mean()
print("Using logerroravg = ", logerroravg)

('Using logerroravg = ', 0.011457219606756525)


In [546]:
X_all = properties

In [5]:
# Add taxrate
X_all['taxrate'] = np.divide(X_all['taxamount'],X_all['taxvaluedollarcnt'])

In [8]:
# Add roomcnt_offby
X_all['roomcnt_offby'] = np.where(
                                X_all['roomcnt']==0,
                                0,
                                X_all['roomcnt'].add(X_all['bathroomcnt'])
                            )

In [9]:
# Clean features
for col in train:
    dt = X_all[col].dtype
    if dt == 'float64':
        X_all[col].fillna(-1, inplace='True')  
    elif dt == 'object':
        X_all = pd.get_dummies(X_all,columns=[col], dummy_na=True)

In [534]:
Y_all = model.predict(X_all)

In [ ]:
results = pd.DataFrame(index=X_all.index)
results.index.names = ['ParcelId']
results['201610'] = Y_all
results['201611'] = Y_all
results['201612'] = Y_all
results['201710'] = Y_all
results['201711'] = Y_all
results['201712'] = Y_all

In [ ]:
submission = submission.drop(submission.columns[0:], axis=1)
submission = submission.join(results)

# Use average for properties with missing data
submission.fillna(logerroravg, inplace=True)       
submission.columns

In [ ]:
# Round as per rules
submission = submission.round(4)

## Sanity Checks

In [ ]:
# Average log error
submission.describe().round(4)

In [ ]:
# Check for NaN
submission.isnull().sum()

In [ ]:
# Check if any duplicates
submission[submission.index.duplicated(keep=False)]

In [ ]:
# Check additional values in submission file
submission[~submission.index.isin(properties.index)]

In [ ]:
# Check additional values in properties file
properties[~properties.index.isin(submission.index)]

## Dump File

In [ ]:
# Write file
submission_f = '/Users/Jose/Desktop/Zillow/submission.csv'
submission_fh = open(submission_f, 'wb')
submission.to_csv(submission_fh, sep=',', header='true')
submission_fh.close()

## Historical Records

## Resources

In [ ]:
# bedroomcnt
#plt.trainlabel('bedroomcnt')
#plt.ylabel('logerror')
#plt.scatter(train['bathroomcnt'], train['logerror'], 
#           alpha=1, s=2, color='r')
#plt.atrainis([0, 10,-5, 5])
#plt.show()

#date
#plt.trainlabel('transactiondate')
#plt.ylabel('logerror')
#plt.scatter(train['bedroomcnt'], train['logerror'], alpha=1, s=2, color='r')
#plt.atrainis(['2016-01-01', '2017-01-01',-5, 5])
#plt.gcf().autofmt_traindate()
#plt.show()

#import matplotlib as mpl
#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train['longitude'], train['latitude'], c=train['logerror'], s=20, lw=0, cmap='seismic')
#plt.atrainis([min(train['longitude']), matrain(train['longitude']),min(train['latitude']), matrain(train['latitude'])])
#plt.colorbar()
#plt.show()

#train2 = train.query('logerror < -2.')

#train2 = train.query('roomcnt == 0 ')

#train2 = train.query('bedroomcnt > 2')

#train2 = train.sample(frac=.10)

#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train2['longitude'], train2['latitude'], c=train2['logerror'], s=20, lw=0, cmap='seismic')
#plt.colorbar()
#plt.show()